# **Assignment - 6**

**202011050(Krishna Savaliya)**

In [ ]:
#imported all required libraries here
import os
import re
import math
import glob
import numpy as np
from numpy.linalg import norm
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import collections
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocessing 

In [ ]:
class Preprocessing:
  
  #To Load the Data Available for applying preprocessing on it
  def load_data(self):
    ##As the data has been extracted ,we can no easily load the data in to the RAM.
    global doc
    global total_doc
    total_doc = 0
    doc = list()                                     #contains real document id list
    
    data=list()                                      #contains list of list of data for each file .each list within list has one corresponding file data
    files = list()                                   #contains list of all file names in the root directory and its subdirectoris  
    location = "/content/drive/MyDrive/FIRE_Dataset_EN_2010"        #root directory
    # r-root, d-directories, f-files
    for r, d, f in os.walk(location):
      for item in f:
        if '.utf8' in item:                         #getting only ".utf8" files ..ignoring query files as we will not fit the query files
          files.append(os.path.join(r, item))
    
     
    for name in files:                               #traversing all the ".utf8" files and will be extracting the data from it in the ram 
      #check if name contains index type files or not
      if total_doc % 5000 == 0:
        print("record loaded = ",total_doc)
      check = re.findall("index",str(name))
      if len(check) != 0:  #contains index then continue and ignore this file
         continue
      #i have directly loaded all the files in to the colab so ignore some files while listing the files
      if name == '.config' or name == 'sample_data':
        continue
      else:
        f = open(name,'r')
        soup = BeautifulSoup(f,"html.parser")
        required_data = soup.find("text")
        temp = required_data.text
        #check if document contains any words or not
        check = re.findall("[A-Za-z0-9]",temp)
        if len(check) == 0:
          continue
        data.append(temp)       
        doc_id = soup.find("docno")
        doc.append(doc_id)
        total_doc += 1
        
    return data
    
    #function for removing numerical and punctuation from data
    #input - data from which numerical and punctuation has to be removed
    #output -  processed data
 
  def preprocess(self,data):
      #retrieving each doc data and then removing numerical and punctuation from each document
      stop_words = stopwords.words('english')
      lemmatizer = WordNetLemmatizer()
      preprocessed_data=list()
      for doc_text in data:
        
        #removing numerical and punctuation
        
        doc_text = re.sub(r'[^\w\s]', '',doc_text)
        doc_text = re.sub('\d','',doc_text)
        

        #tokenizing
        data_tokenized = nltk.word_tokenize(doc_text)
       
        
        #converting to smallcase
        data_smallcase = [word.lower() for word in data_tokenized] 
        
        
        #removing stopwords
        data_without_stopwords = list()       
        for word in data_smallcase:
          if word not in stop_words:
            data_without_stopwords.append(word)
        
        ## Lemmatization
        #print(data_without_stopwords) 
        prev_data = list() 
        for word in data_without_stopwords:
          prev_data.append(lemmatizer.lemmatize(word,pos='v'))
        preprocessed_data.append(prev_data)
                 
        
      return preprocessed_data
  
  def extractUnique(self,data):
    #return unique feature from the entire corpus
    return collections.Counter([x for sublist in data for x in sublist])
     

#Custom TFIDF Class

In [ ]:
class TFIDF:
    
  def __init__(self):
    
    self.idf_matrix = csr_matrix(self.find_idf_matrix1())
    
  #input -  documents list
  #output - return term document matrix
  def fit(self,documents):
    global feature_list
    #indices for generating sparse matrix  ##
    global document_feature_list
    row = list()
    column = list() 
    value = list() 
      
    reverse_feature_list = dict()
    for i,word in enumerate(feature_list):
      reverse_feature_list[word] = i 
    
    for i,document in zip(range(total_doc),documents):
      total_words_in_doc = len(document)
      if i%10000 == 0 :
        print("document processed=",i)
      
      for word in document_feature_list[i]:
        count = document_feature_list[i][word]
        j = reverse_feature_list[word]
        row.append(i)
        column.append(j)
        value.append(float(count)/total_words_in_doc)       #term frequency
      
    print("creating sparse matrix")
    tf_sparse = csr_matrix((value, (row, column)), shape=(total_doc, len(feature_list)))
    print("processed now returning")
    return csr_matrix.multiply(tf_sparse,self.idf_matrix)       #tf*idf 


  def transform(self,query):
    #transformes the query to tf-idf representation
    global feature_list 
    
    ##indices for generating sparse matrix  ##

    row = list()
    column = list() 
    value = list() 
      
     
    
    total_words_in_doc = len(query)
    
    for i,word in zip(range(len(feature_list)),feature_list):
        count = query.count(word)
        element_value = float(count)/total_words_in_doc
        row.append(0)
        column.append(i)
        value.append(element_value) 
    tf_sparse = csr_matrix((value, (row, column)), shape=(1, len(feature_list)))
    return csr_matrix.multiply(tf_sparse,self.idf_matrix)

  def return_feature_generator(self):
    global feature_list 
    for ele in feature_list: 
      yield (ele)
      
  def find_idf_matrix1(self):
    #return list which stores the idf value of each unique term
    global feature_list
    global feature_collection 
    global total_doc
    global data_preprocessed 
    print("Buiding IDF Matrix\n")
    
    
    temp_matrix = np.zeros((1,len(feature_list)))
    feature_list_generator = self.return_feature_generator()

    for i,ele in zip(range(len(feature_list)),feature_list_generator):
      temp_matrix[0,i] = feature_collection[ele]
      
    print("\n Creating sparse IDF Matrix")  
    print("processed now returning")
    return np.log10(np.multiply(total_doc,np.reciprocal(temp_matrix)))
    

          
        

        
      
    
    
    
    
    
    
  

#Feature Extraction 

In [ ]:
##generating object for preprocesssing object as previous generator has already been used 


#obj = Preprocessing()
#data = obj.load_data()
#data_preprocessed = obj.preprocess(data_preprocessed)

infile = open('/content/drive/MyDrive/dump/corpus_text_tokens','rb')
data_preprocessed = pickle.load(infile)
infile.close()

total_doc = len(data_preprocessed)
 
print("Number of documents=",total_doc)

#print(data_preprocessed)

print("fetching all the unique words in the corpus using above generator objects..\n")
obj = Preprocessing()
feature_collection = obj.extractUnique(data_preprocessed)
feature_list= list(feature_collection)
document_feature_list = list()
for document in data_preprocessed:
  c = collections.Counter([word for word in document])
  document_feature_list.append(c)

print("preprocessing completed for extracting feature_list..\n")

print("No of features in the corpus=",len(feature_list))


obj_tfidf = TFIDF()



Number of documents= 125516
fetching all the unique words in the corpus using above generator objects..

preprocessing completed for extracting feature_list..

No of features in the corpus= 306069
Buiding IDF MATRIX......


 Creating sparse IDF MATRIX
processed now returning


#TFTDF Matrix 

In [ ]:
#passing preprocessed generator
term_doc_matrix = obj_tfidf.fit(data_preprocessed)

print(term_doc_matrix.shape)
#generated object for term_doc_matrix generator

#cannot print shape because the class is generator
 
del feature_collection 
del document_feature_list
print("object for term_doc_matrix_generated generator is generated !!!")

document processed= 0
document processed= 10000
document processed= 20000
document processed= 30000
document processed= 40000
document processed= 50000
document processed= 60000
document processed= 70000
document processed= 80000
document processed= 90000
document processed= 100000
document processed= 110000
document processed= 120000
creating sparse matrix....
processed now returning.....
(125516, 306069)
object for term_doc_matrix_generated generator is generated !!!


#Transforming Query

In [ ]:
from bs4 import BeautifulSoup
#fetching the queries from /content/drive/MyDrive/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt

QUERIES = list()                               #will contain all the preprocessed queries 
query_ids = list()                             #will contain all the query ids
f = open("/content/drive/MyDrive/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt")
soup = BeautifulSoup(f,"html.parser")
queries = soup.find_all("top")

for query_data in queries:
  query_ids.append(query_data.num.text)
  QUERIES.append(query_data.text)                            #fetching query and inserting to query list


#preprocessing the query

#Preprocessing to get QUERY GENERATOR
QUERY_GENERATOR = obj.preprocess(QUERIES)

#Preprocessing of Query Complete

#Transforming Queries
transformed_query = list() 
for query in QUERY_GENERATOR:
  transformed_query.append(obj_tfidf.transform(query))

print("Queries Transform with sparse representation completed......")
 
#Transformation Complete


Queries Transform with sparse representation completed......


# **Utility Function**

# MAP Calculation

In [ ]:
#VECTOR_MATRIX is the document converted to vectors using arbitrary represntation 
# tfidf / word2vec etc
def calculate_map(modified_transformed_query,VECTOR_MATRIX):       
  
  def find_similarity(transformed_query,VECTOR_MATRIX):
  
    
    query_cosine_similarity = list()  
    for query in transformed_query:
      #iterating among all document in the term-document matrix
      query_cosine_similarities = cosine_similarity(query, VECTOR_MATRIX)
      query_cosine_similarity.append(list(query_cosine_similarities[0,:]))
    return query_cosine_similarity

  query_cosine_similarity = find_similarity(modified_transformed_query,VECTOR_MATRIX)


  ##above is query similarity generator


  #print(len(query_cosine_similarities))
  ##fetching top 10 document for each query

  import pickle 
  f = open("/content/drive/MyDrive/dump/file_names","rb")
  file_names = pickle.load(f)
  
  column = ["RANK","DOCUMENT NAME","cosine similarity score"]                      # columns for our panda data frame
  ranked_queries_doc = list()
  rank_len = 10
  k=0                    
  for query_similarities in query_cosine_similarity: 
    ranked_doc = list()
    for id,score in enumerate(query_similarities):
      ranked_doc.append([id,score])

  
    ##greater the cosine similarity higher the rank of document
    ranked_doc = sorted(ranked_doc,key = lambda ele:ele[1] ,reverse=True)[:rank_len]
    ranked_doc = [[i+1,file_names[ele[0]],ele[1]] for i,ele in  zip(range(len(ranked_doc)),ranked_doc)]
    ranked_queries_doc.append(ranked_doc)
    data_frame = pd.DataFrame(data = ranked_doc,columns = column)
    print("\n")
    print("query ID= ",query_ids[k])
    print("\n")
    k = k + 1
    display(data_frame.head(10))


  #ranked_queries_doc contains list of list where each list is of type [docname,cosine_similarity score]


  query_doc_relevant = dict()

  #fetching the document v/s relevance from  /content/drive/MyDrive/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt  for each query

  f = open("/content/drive/MyDrive/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt","r")
  lines = f.readlines()
  for line in lines:
    line_split = line.split(" ")
    line_split[3] =  line_split[3].strip()
    query_no = line_split[0]
    doc_name = line_split[2]
    rel_score = line_split[3]
  
  
    if query_no == " ":
      continue 
    else:
      result = query_doc_relevant.get(query_no,0)
      if result == 0:
        query_doc_relevant[query_no] = [[doc_name,rel_score]]
      else:
        query_doc_relevant[query_no].append([doc_name,rel_score]) 


  queries_precision = list()


  for no,query in zip(range(len(ranked_queries_doc)),ranked_queries_doc):
    #query contains [[index,docname,cosine_similarity],[..],...]
    list_fetched_doc = [ ele[1] for ele in query] 
    #print(query_doc_relevant[query_ids[no]][0][0]," ",query_doc_relevant[query_ids[no]][0][1])
    
    ##counting no of correctly retrieved document by filtering all the document which is fetched correctly by system  
    ground_query = query_doc_relevant[query_ids[no]]
    count = 0 
    for ground in ground_query:
        if ground[0] in list_fetched_doc:
          if ground[1] == "1":
            count = count + 1 
   
    queries_precision.append(float(count)/10) 

  return float(sum(queries_precision))/len(queries_precision)

  


#**Q1 Implementing Rocchio Algorithm**

In [ ]:
##takes transformed query as input and returns modifed query using rocchio equation


def rocchio(transformed_query,VECTOR_MATRIX):
  query_doc_relevant = dict()

  #fetching the document v/s relevance from  /content/drive/MyDrive/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt  for each query

  f = open("/content/drive/MyDrive/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt","r")
  lines = f.readlines()
  for line in lines:
    line_split = line.split(" ")
    line_split[3] =  line_split[3].strip()
    query_no = line_split[0]
    doc_name = line_split[2]
    rel_score = line_split[3]
  
  
    if query_no == " ":
      continue 
    else:
      result = query_doc_relevant.get(query_no,0)
      if result == 0:
        query_doc_relevant[query_no] = [[doc_name,rel_score]]
      else:
        query_doc_relevant[query_no].append([doc_name,rel_score]) 
  
  ###seperating relevant and non relevant document for each query

  #building file name v/s indices matrix
  import pickle 
  f = open("/content/drive/MyDrive/dump/file_names","rb")
  file_names = pickle.load(f)
  file_dict = dict()
  for indices,value in enumerate(file_names):
    file_dict[value] = indices 


  ##modified queries
  modified_transformed_query= list()
  i=0

  alpha=1
  beta=0.75
  gamma=0.15
  for ele,value in query_doc_relevant.items():
    no_of_rel=0
    no_of_irrel = 0
    rel_doc_sum = np.zeros((1,VECTOR_MATRIX.shape[1]))
    irrel_doc_sum = np.zeros((1,VECTOR_MATRIX.shape[1]))
    for rel_doc in value:
      if rel_doc[1] == "0":
        pass
        #irrel_doc_sum += term_doc_matrix[file_dict.get(rel_doc[0]),:]
        #no_of_irrel = no_of_irrel + 1
      else:
        rel_doc_sum += VECTOR_MATRIX[file_dict.get(rel_doc[0]),:]
        no_of_rel = no_of_rel + 1
  
    qm = alpha*transformed_query[i] + beta*(1/no_of_rel)*rel_doc_sum - gamma * (1/(VECTOR_MATRIX.shape[0]-no_of_rel)) * (np.sum(VECTOR_MATRIX,axis=0) - rel_doc_sum);
   
    modified_transformed_query.append(qm) 
    i = i + 1
  return modified_transformed_query



#**Q4  Query Expansion and Trasformation**  

In [ ]:
def expand_transform_query(model,model_obj):
  global QUERY_GENERATOR
  Query_expanded = list()
  for query in QUERY_GENERATOR:
    temp = list()
    for tok in query:
      syns = wordnet.synsets(tok)
      syns = syns[0:10]
      for synonym in syns:
        temp.append(synonym.lemmas()[0].name())
  
    Query_expanded.append(temp)

  print("expansion complete")


  #Preprocessing of Query Complete

  #Transforming Queries
  if model == "tfidf":
    expanded_transformed_query = list() 
    for query in Query_expanded:
      expanded_transformed_query.append(model_obj.transform(query))
  else:
    expanded_transformed_query = list()
    for i in Query_expanded:
      
      temp = np.zeros((350))
      for j in i:
        try:
          temp = temp + model_obj[j]
        except:
          continue
      expanded_transformed_query.append(temp)
  print("transformation complete")  
  return expanded_transformed_query





# **Using Custom TFIDF Representation**

#**Q2 TFIDF + ROCCHIO**

In [ ]:
modified_transformed_query = rocchio(transformed_query,term_doc_matrix)

calculating map

In [ ]:
map = calculate_map(modified_transformed_query,term_doc_matrix)



query ID=  76




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070603_nation_story_7869357.utf8,0.320252
1,2,1070602_nation_story_7865940.utf8,0.319014
2,3,1070602_nation_story_7865944.utf8,0.306179
3,4,1070611_nation_story_7906812.utf8,0.239266
4,5,1070607_opinion_story_7886010.utf8,0.228050
5,6,1070530_nation_story_7849973.utf8,0.218302
6,7,1070605_frontpage_story_7877876.utf8,0.154808
7,8,1070531_frontpage_story_7855182.utf8,0.148881
8,9,1050409_frontpage_story_4593939.utf8,0.141898
9,10,1070603_frontpage_story_7869833.utf8,0.135834




query ID=  77




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060727_foreign_story_6531510.utf8,0.231339
1,2,1070807_nation_story_8160445.utf8,0.230443
2,3,1060814_foreign_story_6609200.utf8,0.210073
3,4,1060724_opinion_story_6493371.utf8,0.206717
4,5,1060717_foreign_story_6488684.utf8,0.203178
5,6,1060721_foreign_story_6506359.utf8,0.201470
6,7,1060724_foreign_story_6517899.utf8,0.200246
7,8,1060821_opinion_story_6621357.utf8,0.199443
8,9,1060714_foreign_story_6477224.utf8,0.197429
9,10,1060810_foreign_story_6592351.utf8,0.191717




query ID=  78




...

,RANK,DOCUMENT NAME,cosine similarity score
0,1,1041110_foreign_story_3987311.utf8,0.562285
1,2,1041030_nation_story_3943294.utf8,0.541744
2,3,1041203_opinion_story_4069612.utf8,0.516117
3,4,1041106_foreign_story_3972071.utf8,0.511056
4,5,1041105_foreign_story_3968081.utf8,0.506820
5,6,1041108_foreign_story_3978518.utf8,0.504124
6,7,1041112_frontpage_story_3995357.utf8,0.500263
7,8,1041030_foreign_story_3943345.utf8,0.486986
8,9,1041111_frontpage_story_3991738.utf8,0.479890
9,10,1041112_foreign_story_3995445.utf8,0.467618




query ID=  124




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1040905_opinion_story_3718881.utf8,0.365155
1,2,1050323_opinion_story_4467789.utf8,0.353493
2,3,1070130_calcutta_story_7314924.utf8,0.312621
3,4,1040918_business_story_3772991.utf8,0.304697
4,5,1060117_opinion_story_5727653.utf8,0.304038
5,6,1060818_business_story_6624384.utf8,0.284586
6,7,1060316_business_story_5971953.utf8,0.282736
7,8,1050217_nation_story_4389677.utf8,0.278004
8,9,1060607_business_story_6320638.utf8,0.277501
9,10,1060924_business_story_6786542.utf8,0.269702




query ID=  125




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070806_foreign_story_8155233.utf8,0.417170
1,2,1070712_opinion_story_8046139.utf8,0.353002
2,3,1070728_foreign_story_8117144.utf8,0.350625
3,4,1070822_foreign_story_8222596.utf8,0.332761
4,5,1070729_foreign_story_8120955.utf8,0.332597
5,6,1070711_foreign_index.utf8,0.322778
6,7,1070705_foreign_story_8019084.utf8,0.312443
7,8,1070726_foreign_story_8108245.utf8,0.308380
8,9,1070731_foreign_story_8128927.utf8,0.307852
9,10,1070520_opinion_story_7801150.utf8,0.287856


#MAP TFIDF + ROCCHIO

In [ ]:
print("map with tfidf + rocchio = ",map)

map with tfidf + rocchio =  0.43000000000000016


#**Q3 TFIDF + ROCCHIO + EXPANSION**      

here we are expanding the initial query and after that we are applying rocchio algorithm

In [ ]:

##reading dumped expanded_transformed_query 

expanded_transformed_query = expand_transform_query("tfidf",obj_tfidf)
#expanded_transformed_query = pickle.load(open("/content/drive/MyDrive/expanded_transformed_query","rb"))

modified_transformed_query = rocchio(expanded_transformed_query,term_doc_matrix)       ##returns modified query


expansion complete
transformation complete


calculating map

In [ ]:
map = calculate_map(modified_transformed_query,term_doc_matrix)




query ID=  76




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070602_nation_story_7865940.utf8,0.274605
1,2,1070603_nation_story_7869357.utf8,0.273596
2,3,1070611_nation_story_7906812.utf8,0.243645
3,4,1070530_nation_story_7849973.utf8,0.235583
4,5,1070602_nation_story_7865944.utf8,0.214263
5,6,1070607_opinion_story_7886010.utf8,0.203086
6,7,1070530_nation_story_7849974.utf8,0.198026
7,8,1070531_frontpage_story_7855182.utf8,0.195212
8,9,1070605_frontpage_story_7877876.utf8,0.184459
9,10,1070603_frontpage_story_7869833.utf8,0.178850




query ID=  77




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060717_foreign_story_6488684.utf8,0.331339
1,2,1060810_calcutta_story_6554869.utf8,0.313908
2,3,1070727_nation_story_8112080.utf8,0.305197
3,4,1051229_foreign_story_5657775.utf8,0.302409
4,5,1060712_nation_story_6467796.utf8,0.296162
5,6,1060806_foreign_story_6575214.utf8,0.295523
6,7,1060814_foreign_story_6609200.utf8,0.294848
7,8,1060719_foreign_story_6496938.utf8,0.292755
8,9,1060830_calcutta_story_6663465.utf8,0.290571
9,10,1051229_nation_story_5657738.utf8,0.289047




query ID=  78




...

,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070608_opinion_story_7891152.utf8,0.272764
1,2,1050116_bengal_story_4259941.utf8,0.268012
2,3,1051221_frontpage_story_5627402.utf8,0.266925
3,4,1050128_nation_story_4305602.utf8,0.231268
4,5,1060401_nation_story_6041679.utf8,0.221690
5,6,1050304_foreign_story_4452390.utf8,0.211465
6,7,1061115_opinion_story_6979498.utf8,0.200576
7,8,1041112_nation_story_3995294.utf8,0.199196
8,9,1041208_opinion_story_4086626.utf8,0.196514
9,10,1060717_nation_story_6488521.utf8,0.194504




query ID=  124




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1040905_opinion_story_3718881.utf8,0.305297
1,2,1050323_opinion_story_4467789.utf8,0.301433
2,3,1060117_opinion_story_5727653.utf8,0.252344
3,4,1060612_nation_story_6340989.utf8,0.250155
4,5,1040918_business_story_3772991.utf8,0.249002
5,6,1060316_business_story_5971953.utf8,0.235934
6,7,1060607_business_story_6320638.utf8,0.233262
7,8,1060818_business_story_6624384.utf8,0.231323
8,9,1060212_nation_story_5836521.utf8,0.227401
9,10,1050217_nation_story_4389677.utf8,0.224701




query ID=  125




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070705_foreign_story_8019084.utf8,0.253414
1,2,1070716_foreign_story_8064283.utf8,0.247723
2,3,1070728_foreign_story_8117144.utf8,0.246932
3,4,1070705_foreign_story_8019086.utf8,0.244387
4,5,1070822_foreign_story_8222596.utf8,0.243082
5,6,1070729_foreign_story_8120955.utf8,0.236545
6,7,1070630_foreign_story_7995955.utf8,0.234654
7,8,1070726_foreign_story_8108245.utf8,0.228282
8,9,1070806_foreign_story_8155233.utf8,0.217813
9,10,1070521_foreign_story_7806832.utf8,0.215072


#MAP ROCCHIO + EXPANSION

In [ ]:
print("map for tfidf +  rocchio + query expansion = ",map)

map for tfidf +  rocchio + query expansion =  0.40399999999999997


#**Using Word2Vec**

In [ ]:
#del term_doc_matrix
# upgrade and install gensim
!pip install --upgrade gensim
from gensim.models import Word2Vec

     |████████████████████████████████| 24.2MB 1.6MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
# train CBOW model using corpus text tokens
CBOW_model = Word2Vec(sentences=data_preprocessed, size=350, window=10, min_count=1, workers=4, sg=0, hs=1, negative=0, iter=5)

In [ ]:
# save CBOW model in the desk for future use
with open("/content/drive/MyDrive/Assignment6/CBOW_model", "wb") as fp:   #Pickling
    pickle.dump(CBOW_model, fp)

In [ ]:
# load CBOW model from the disk
with open('/content/drive/MyDrive/Assignment6/CBOW_model','rb') as file:
  CBOW_model = pickle.load(file)

In [ ]:
VECTOR_MATRIX = np.zeros((len(data_preprocessed),350))
#query_array = []
count = 0
for i in data_preprocessed:
  temp = np.zeros((350))
  for j in i:
      temp = temp + CBOW_model[j]
  VECTOR_MATRIX[count,:] = temp
  count = count + 1 
print("length of document array:",len(VECTOR_MATRIX))
'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


length of document array: 125516


'\n#query array\nfor i in QUERY_GENERATOR:\n  temp = np.zeros((350,),dtype=float)\n  for j in i:\n      try:\n        temp = temp + CBOW_model[j]\n      except:\n        continue\n  query_array.append(temp)\nprint("length of query array:",len(query_array))\n'

In [ ]:
# save vector_matrix model in the disk for future use
with open("/content/drive/MyDrive/Assignment6/VECTOR_MATRIX", "wb") as fp:   #Pickling
    pickle.dump(VECTOR_MATRIX, fp)

#**Q5 WORD2VEC + ROCCHIO**

transforming query using cbow model

In [ ]:
#query array
query_array = list() 
for i in QUERY_GENERATOR:
  temp = np.zeros((350))
  for j in i:
      try:
        temp = temp + CBOW_model[j]
      except:
        continue
  query_array.append(temp)
print("length of query array:",len(query_array))


length of query array: 50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


fetching modified rocchio query

In [ ]:

modified_transformed_query = rocchio(query_array,VECTOR_MATRIX)

calculating map for the system

In [ ]:
map = calculate_map(modified_transformed_query,VECTOR_MATRIX)
print(map)



query ID=  76




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070530_nation_story_7849973.utf8,0.868863
1,2,1070603_nation_story_7869357.utf8,0.845879
2,3,1070611_nation_story_7906812.utf8,0.807640
3,4,1070531_frontpage_story_7855182.utf8,0.776296
4,5,1070604_frontpage_story_7872841.utf8,0.769010
5,6,1070531_opinion_story_7852019.utf8,0.765815
6,7,1070625_nation_story_7970372.utf8,0.759816
7,8,1070605_nation_story_7877641.utf8,0.758689
8,9,1070602_nation_story_7865940.utf8,0.747541
9,10,1070605_frontpage_story_7877876.utf8,0.746485




query ID=  77




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060720_foreign_story_6501311.utf8,0.966242
1,2,1060718_foreign_story_6491617.utf8,0.964281
2,3,1060814_foreign_story_6609200.utf8,0.963425
3,4,1060806_foreign_story_6575214.utf8,0.962531
4,5,1060719_foreign_story_6496938.utf8,0.960623
5,6,1060717_foreign_story_6488684.utf8,0.958491
6,7,1060716_foreign_story_6485226.utf8,0.956812
7,8,1060715_foreign_story_6482002.utf8,0.954693
8,9,1060714_foreign_story_6477224.utf8,0.952724
9,10,1060803_foreign_story_6562281.utf8,0.945499




query ID=  78




...



query ID=  124




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060612_nation_story_6340993.utf8,0.854347
1,2,1060612_calcutta_story_6317909.utf8,0.808795
2,3,1061208_calcutta_story_7108893.utf8,0.764902
3,4,1061227_nation_story_7189167.utf8,0.754823
4,5,1060612_nation_story_6340989.utf8,0.749710
5,6,1050912_nation_story_5227211.utf8,0.739808
6,7,1070526_nation_story_7832104.utf8,0.731490
7,8,1060715_calcutta_story_6478430.utf8,0.730027
8,9,1060829_nation_story_6670663.utf8,0.725881
9,10,1070723_calcutta_story_8094302.utf8,0.725122




query ID=  125




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070705_foreign_story_8019084.utf8,0.905825
1,2,1070709_foreign_story_8034677.utf8,0.897257
2,3,1070726_foreign_story_8108245.utf8,0.896418
3,4,1070630_foreign_story_7995955.utf8,0.889659
4,5,1070705_foreign_story_8019086.utf8,0.888826
5,6,1070708_foreign_story_8030768.utf8,0.888603
6,7,1070707_foreign_story_8027755.utf8,0.888513
7,8,1070731_foreign_story_8128927.utf8,0.884241
8,9,1070704_foreign_story_8012634.utf8,0.878486
9,10,1070706_foreign_story_8023448.utf8,0.867583


0.5200000000000001


#**MAP Result**

In [ ]:
print("map without expansion + word2vec + rocchio = ",map)

map without expansion + word2vec + rocchio =  0.5200000000000001


#**Q6 ROCCHIO + EXPANSION + WORD2VEC**



EXPANDING QUERY 

In [ ]:
expand_transformed_query = expand_transform_query("cbow",CBOW_model)


expansion complete
transformation complete


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Applying rochio

In [ ]:
modified_transformed_query = rocchio(expand_transformed_query,VECTOR_MATRIX)

calculating map

In [ ]:
map = calculate_map(modified_transformed_query,VECTOR_MATRIX)



query ID=  76




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070531_opinion_story_7852019.utf8,0.737476
1,2,1070530_nation_story_7849973.utf8,0.717149
2,3,1070602_opinion_story_7862142.utf8,0.689695
3,4,1070605_nation_story_7877641.utf8,0.689580
4,5,1070625_nation_story_7970372.utf8,0.678598
5,6,1070603_nation_story_7869357.utf8,0.675555
6,7,1070531_frontpage_story_7855182.utf8,0.673104
7,8,1070607_opinion_story_7886010.utf8,0.670076
8,9,1070611_nation_story_7906812.utf8,0.669088
9,10,1040914_frontpage_story_3753488.utf8,0.665633




query ID=  77




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060806_foreign_story_6575214.utf8,0.876880
1,2,1060717_foreign_story_6488684.utf8,0.874240
2,3,1060719_foreign_story_6496938.utf8,0.869273
3,4,1060814_foreign_story_6609200.utf8,0.865846
4,5,1060716_foreign_story_6485226.utf8,0.865209
5,6,1060727_foreign_story_6531510.utf8,0.863189
6,7,1060718_foreign_story_6491617.utf8,0.857976
7,8,1060802_foreign_story_6557944.utf8,0.856047
8,9,1051229_foreign_story_5657775.utf8,0.853775
9,10,1060720_foreign_story_6501311.utf8,0.852434




query ID=  78




...

,RANK,DOCUMENT NAME,cosine similarity score
0,1,1041208_opinion_story_4086626.utf8,0.743436
1,2,1041110_foreign_story_3987311.utf8,0.740296
2,3,1041106_foreign_story_3972071.utf8,0.729239
3,4,1041105_foreign_story_3968081.utf8,0.716500
4,5,1041112_frontpage_story_3995357.utf8,0.712618
5,6,1041108_foreign_story_3978518.utf8,0.701325
6,7,1041112_foreign_story_3995445.utf8,0.694690
7,8,1041123_foreign_story_4038063.utf8,0.692819
8,9,1041129_foreign_story_4062221.utf8,0.688494
9,10,1041112_nation_story_3995294.utf8,0.678457




query ID=  124




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1060612_nation_story_6340993.utf8,0.676356
1,2,1060612_nation_story_6340989.utf8,0.624621
2,3,1060612_calcutta_story_6317909.utf8,0.620229
3,4,1061208_calcutta_story_7108893.utf8,0.597605
4,5,1050103_calcutta_story_4203336.utf8,0.595733
5,6,1041124_nation_story_4041778.utf8,0.592708
6,7,1050209_nation_story_4350796.utf8,0.588892
7,8,1050704_calcutta_story_4945535.utf8,0.588510
8,9,1050307_nation_story_4462654.utf8,0.584637
9,10,1050915_calcutta_story_5237801.utf8,0.582245




query ID=  125




,RANK,DOCUMENT NAME,cosine similarity score
0,1,1070630_foreign_story_7995955.utf8,0.839450
1,2,1070708_foreign_story_8030768.utf8,0.832444
2,3,1070709_foreign_story_8034677.utf8,0.828839
3,4,1070705_foreign_story_8019086.utf8,0.828403
4,5,1070705_foreign_story_8019084.utf8,0.824575
5,6,1070707_foreign_story_8027755.utf8,0.822993
6,7,1070520_foreign_story_7803534.utf8,0.814512
7,8,1070704_foreign_story_8012634.utf8,0.814137
8,9,1070706_foreign_story_8023448.utf8,0.811005
9,10,1070726_foreign_story_8108245.utf8,0.802681


#**MAP Result**

In [ ]:
print("map with rochio + query expansion + word2vec = ",map)

map with rochio + query expansion + word2vec =  0.41199999999999987


#Final Results


USING TFIDF

1.   USING ROCCHIO, MAP = 0.4300
2.   ROCCHIO + EXPANSION , MAP = 0.4039 

USING WORD2VEC

1.   USING ROCCHIO, MAP = 0.52
2.   ROCCHIO + EXPANSION, MAP = 0.4119999



